# 混洗（Shuffling）机制

In [ ]:
#改进部分
import numpy as np
x = np.array([-2,-1,0,1,2])
np.where(x > 0, 1.0, -2.0)
#np.where类似于对于向量的if else
#1+e^{-|x|}
#softplus 函数 log(1+e^x)
#问题2 在写Logistic回归中的目标函数时，log（rhoi）可以由softplus函数直接表示

重新划分分区后，数据的顺序被打乱

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("Shuffling").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[*] appName=Shuffling>


创建一个简单的 RDD：

In [3]:
import string

rdd = sc.parallelize(string.ascii_uppercase, numSlices=5)
print(rdd.collect())
#这里存在内存中，故可直接自定义分区
#若是外部文件则自动分区

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [4]:
rdd.getNumPartitions()

5

将每个分区的字母合并成一个字符串，从而可以知道分区是如何划分的：

In [7]:
def concat_letters(iter):
    yield "".join(iter)

rdd.mapPartitions(concat_letters).collect()

['ABCDE', 'FGHIJ', 'KLMNO', 'PQRST', 'UVWXYZ']

将 RDD 重新分区，可以看出来字母的顺序产生了变化：

In [8]:
rdd2 = rdd.repartition(6)
print(rdd2.collect())

['A', 'B', 'C', 'D', 'E', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'K', 'L', 'M', 'N', 'O', 'F', 'G', 'H', 'I', 'J']


In [9]:
rdd2.mapPartitions(concat_letters).collect()

['', 'ABCDE', 'PQRSTUVWXYZ', '', 'KLMNO', 'FGHIJ']

故须关注自己的算法是否与数据顺序相关

使用 `toDebugString()` 查看 RDD 包含的操作：

In [10]:
print(rdd.toDebugString().decode("UTF-8"))
print("\n\n")
print(rdd2.toDebugString().decode("UTF-8"))

(5) ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []



(6) MapPartitionsRDD[8] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[7] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[6] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(5) MapPartitionsRDD[5] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[4] at RDD at PythonRDD.scala:53 []
    |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []


可以看出来 `rdd2` 包含了混洗（shuffling）操作，这也是数据顺序打乱的原因。

如果需要减小分区数目，可以使用 `coalesce()` 函数，避免混洗操作：

In [ ]:
rdd3 = rdd.coalesce(numPartitions=2, shuffle=False)
print(rdd3.collect())
rdd3.mapPartitions(concat_letters).collect()

In [ ]:
print(rdd3.toDebugString().decode("UTF-8"))

如果确实需要增加分区数目，同时希望保持数据顺序，可以在原始 RDD 中增加索引信息：

In [11]:
print(rdd.zipWithIndex().collect())

[('A', 0), ('B', 1), ('C', 2), ('D', 3), ('E', 4), ('F', 5), ('G', 6), ('H', 7), ('I', 8), ('J', 9), ('K', 10), ('L', 11), ('M', 12), ('N', 13), ('O', 14), ('P', 15), ('Q', 16), ('R', 17), ('S', 18), ('T', 19), ('U', 20), ('V', 21), ('W', 22), ('X', 23), ('Y', 24), ('Z', 25)]


增加索引信息，可用于过滤等其他操作

In [ ]:
rdd4 = rdd.zipWithIndex().repartition(5)
print(rdd4.collect())

然后定义一个分区映射函数，在合并数据行时获取分区中第一条数据的索引：

In [ ]:
def concat_letters_with_order(iter):
    letters_and_indices = list(iter)
    letters = map(lambda x: x[0], letters_and_indices)
    indices = map(lambda x: x[1], letters_and_indices)
    if len(letters_and_indices) < 1:
        yield ()
    else:
        first_ind = next(indices)
        combined_letters = "".join(letters)
        yield combined_letters, first_ind

rdd5 = rdd4.mapPartitions(concat_letters_with_order)
print(rdd5.collect())

然后过滤空分区，并按生成的索引对 RDD 进行排序：

In [ ]:
rdd5.filter(lambda x: len(x) > 1).collect()

In [ ]:
rdd5.filter(lambda x: len(x) > 1).sortBy(lambda x: x[1], ascending=True).collect()

In [ ]:
计算结果和观测是否一一对应？是否只是汇总操作？据此判断其排列不变性，从而决策是否考虑混洗。